# Dimensional Modeling in Python

## Import Dependencies

In [1]:
import pandas as pd
import psycopg2
import psycopg2.extras as extras
from db import local_dbc, df_to_table
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

## Reading CSV File

In [2]:
df = pd.read_csv('disclosure_data.csv', encoding='utf-8')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (6,16,19,85,122) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Data Quality Check for Nulls

In [3]:
percent_missing = df.isnull().sum() * 100 / len(df)

missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

percent_missing.head(100)

CASE_NUMBER                          0.000000
CASE_STATUS                          0.000000
RECEIVED_DATE                        0.000000
DECISION_DATE                        0.000000
REFILE                               0.000000
ORIG_FILE_DATE                      99.898957
PREVIOUS_SWA_CASE_NUMBER_STATE      99.896829
SCHD_A_SHEEPHERDER                   0.005318
EMPLOYER_NAME                        0.001064
EMPLOYER_ADDRESS_1                   0.006382
EMPLOYER_ADDRESS_2                  64.691179
EMPLOYER_CITY                        0.001064
EMPLOYER_STATE_PROVINCE              0.002127
EMPLOYER_COUNTRY                     0.025527
EMPLOYER_POSTAL_CODE                 0.004254
EMPLOYER_PHONE                       0.002127
EMPLOYER_PHONE_EXT                  92.026080
EMPLOYER_NUM_EMPLOYEES               0.015954
EMPLOYER_YEAR_COMMENCED_BUSINESS     0.044672
NAICS_CODE                           0.086153
FW_OWNERSHIP_INTEREST                0.007445
EMP_CONTACT_NAME                  

## Cleaning Column Names

In [4]:
df.columns = [col.lower() for col in df.columns]

## Subsetting DataFrame

In [5]:
df = df[['case_number',
'case_status',
'received_date',
'decision_date',
'refile',
'orig_file_date',
'previous_swa_case_number_state',
'schd_a_sheepherder',
'employer_name',
'employer_address_1',
'employer_address_2',
'employer_city',
'employer_state_province',
'employer_country',
'employer_postal_code',
'employer_phone',
'employer_phone_ext',
'employer_num_employees',
'employer_year_commenced_business',
'naics_code',
'fw_ownership_interest',
'emp_contact_name',
'emp_contact_address_1',
'emp_contact_address_2',
'emp_contact_city',
'emp_contact_state_province',
'emp_contact_country',
'emp_contact_postal_code',
'emp_contact_phone',
'emp_contact_email',
'agent_attorney_name',
'agent_attorney_firm_name',
'agent_attorney_phone',
'agent_attorney_phone_ext',
'agent_attorney_address_1',
'agent_attorney_address_2',
'agent_attorney_city',
'agent_attorney_state_province',
'agent_attorney_country',
'agent_attorney_postal_code',
'agent_attorney_email']]



## Create Dimension Tables

In [9]:
### testing bulk insert function


In [6]:
# load csv into df
df = pd.read_csv('disclosure_data.csv', encoding='utf-8')

# connect to db and set cursor
db = local_dbc()
cur = db.cursor()

# set table/columns vars
table = 'disclosure_test.disclosure_test_fact'
cols = ','.join(list(df.columns))

# convert rows into tuples
tuples = [tuple(x) for x in df.to_numpy()]

# set sql query for insert statement
sql = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)

# bulk insert values into table
extras.execute_values(cur, sql, tuples)

# commit changes
db.commit()

In [17]:
tuples

[('A-09245-62715',
  'Denied',
  '9/3/09',
  '4/30/20',
  'N',
  nan,
  nan,
  'N',
  'SATURN ENTERPRISES INC',
  '2340 N WASHTENAW AVE',
  nan,
  'CHICAGO',
  'ILLINOIS',
  'UNITED STATES OF AMERICA',
  '60647',
  '7732769550',
  nan,
  2.0,
  1998.0,
  '23611',
  'N',
  'zygmunt f oparowski',
  '2340 n washtenaw ave',
  nan,
  'chicago',
  'ILLINOIS',
  'UNITED STATES OF AMERICA',
  '60647',
  '7732769550',
  'saturn_enterprises@yahoo.com',
  'Christopher E Kurczaba',
  'Kurczaba Law Offices',
  '7737740011',
  nan,
  '6219 N Milwaukee Ave',
  nan,
  'Chicago',
  'ILLINOIS',
  'UNITED STATES OF AMERICA',
  '60646',
  'kurczabalaw@sbcglobal.net'),
 ('A-10070-89932',
  'Certified',
  '10/22/19',
  '9/4/20',
  'N',
  nan,
  nan,
  'N',
  'SUNRAY ENTERPRISE INC.',
  '3621 VININGS SLOPE SUITE#4310',
  nan,
  'ATLANTA',
  'GEORGIA',
  'UNITED STATES OF AMERICA',
  '30339',
  '678-584-1312',
  '224',
  45.0,
  2002.0,
  '541512',
  'N',
  'Sunitha  Shivaram',
  '3621 Vinings Slope Suite#431

In [187]:
# ****************************************************************** this does not work
# *********** mogrify no longer exists

def do_execute_batch(conn, df, table, page_size=150):
    tups = [tuple(x) for x in df.to_numpy()]

    cols = ','.join(list(df.columns))
    
    sql = "insert into {} VALUES {}".format(table, ', '.join(['%s'] * len(cols))),'id'
    cursor = db.cursor()
    try:
        extras.execute_values(cursor, sql, tups, page_size)
        conn.commit()
        print('Data inserted')
    except psycopg2.DatabaseError as e:
        print(e)
        cursor.close()


